<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Thymio-Project" data-toc-modified-id="Thymio-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Thymio Project</a></span></li><li><span><a href="#Complete-Program" data-toc-modified-id="Complete-Program-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Complete Program</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#define-the-state-machine" data-toc-modified-id="define-the-state-machine-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>define the state machine</a></span><ul class="toc-item"><li><span><a href="#define-the-Robot-object" data-toc-modified-id="define-the-Robot-object-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>define the Robot object</a></span></li><li><span><a href="#Define-the-statefunctions" data-toc-modified-id="Define-the-statefunctions-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Define the statefunctions</a></span></li></ul></li><li><span><a href="#......" data-toc-modified-id="......-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>......</a></span></li><li><span><a href="#........." data-toc-modified-id=".........-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>.........</a></span></li></ul></li><li><span><a href="#........" data-toc-modified-id="........-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>........</a></span></li><li><span><a href="#pathplanning" data-toc-modified-id="pathplanning-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>pathplanning</a></span></li></ul></div>

# Thymio Project







In [ ]:
pip install pyserial

# Complete Program


## Imports
Import all the libraries needed to run the main program

In [ ]:
#import standart libraries
import os
import sys
import time
import serial
import numpy as np
from numpy import linalg as LNG 
import math
import cv2
import matplotlib.pyplot as plt
from matplotlib import colors
from PIL import Image

%matplotlib inline

# Adding the src folder in the current directory as it contains the script
# with the Thymio class and all the files with the group generated functions and classes
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
from Thymio import Thymio
#import functions made by group
from pathPlanning import pathPlaning
import test_vision
#import ANN
#import robot_control
import sys

## define the state machine
In this section the different states are defined as in the state machine graph shown earlier. The functions in this sections are named like the states they represent and they wrap the functions imported from the pathPlanning.py, test_vision.py,ANN.py and robot_control.py. 

### define the Robot object
This object contains all the variables needed in the state machine to make decisions.

In [3]:
class stateNames_t:
    def __init__(self):
        self.goalKnown='goalKnown'
        self.planning='planning'
        self.planAcquired='planAcquired'
        self.newPath='newpath'
        self.checkingPath='checkingPath'
        self.underWay='underWay'
        self.obstacleAvoidance='obstalceAvoidance'
        self.goalReached='goalReached'
class FSMHelper:
    def __init__(self,thymio,kidnappingDistance,currentRobotPosition,equalTolerance):
        self.kidnapDistance=kidnappingDistance
        self.tolerance=equalTolerance
        self.currentPosition=currentRobotPosition

        self.newPositionEstimate=0    
        self.thymio=thymio      
        self.obstacleDetected=False
        self.doStop=False
        self.goalReached=True
        self.goal=np.zeros((2,1))
        self.pathToFollow=np.array([[0],[0]])
        self.FSMStates=stateNames_t()
        self.straightenedImage=0

        

### Define the statefunctions
In this section the different functions used to represent the states of the state machine are defined. Each function takes the FSMHelper object to make the decisions which state is the next.

In [4]:
def goalKnown(robot):
    if(LNG.norm(np.subtract(robot.currentGoal,robot.newGoal))>robot.tolerance):
        robot.doStop=True
        robot.currentGoal=robot.newGoal.copy()     
        return robot.FSMStates.planning
    else:
        return robot.FSMStates.planAccuired
def planning(robot):
    robot.pathPlanner.setGoal(robot.currentGoal)
    return robot.FSMStates.planAccuired
def planAcquired(robot):
    if(LNG.norm(np.subtract(robot.currentPosition,robot.newPositionEstimate))>robot.kidnappingDistance):
        robot.pathPlanner.setStart(robot.newPositionEstimate)
        robot.currentPosition=robot.newPositionEstimate
        return robot.FSMStates.newPath
    else:
        return robot.FSMStates.checkingPath
def newPath(robot):#pathPlanner,pathToFollow
    robot.pathToFollow=robot.pathPlanner.getOptimizedPath()
    return robot.FSMStates.checkingPath
    
def checkingPath(robot):
    if(len(pathToFollow)):
        robot_control.path_following(robot.pathToFollow)
        return robot.FSMStates.underWay
    else:  
        robot.doStop=True
        return robot.FSMStates.goalReached
def underWay(robot):
    if(robot.obstacleDetected==True):
        return robot.FSMStates.obstacleAvoidance
    else:
        return robot.FSMStates.planAcquired
def avoidObstacle(robot):     
    ANN.run_ann_without_memory(robot.thymio)
    return robot.FSMStates.checkingPath 
def goalReached(robot):
    robot.goalReached=True
    
    
#define the concrete stateName object to make the dictionary for the actual state machine
stateName=stateNames_t()
switch = {
    stateName.goalKnown        : goalKnown,
    stateName.planning         : planning,
    stateName.planAcquired     : planAcquired,
    stateName.newPath          : newPath,
    stateName.checkingPath     : checkingPath,
    stateName.underWay         : underWay,
    stateName.obstacleAvoidance: avoidObstacle,
    stateName.goalReached      : goalReached,
}
currentState=stateName.goalKnown
futureState=stateName.goalKnown

## ......
The first steps are to try to connect to the camera and the thymio and then set different decision variables

## .........
In this section the vision part is used to generate a map and then define a pathfinding object for that map. These section represents the 3 first states, since that division can be done in a Notebook

In [5]:
th=Thymio.serial(port="COM5", refreshing_rate=0.1)

In [ ]:
cameraIndex=0             #TODO: change it to the correct camera. currently uses webcam
videoCapture = cv2.VideoCapture(cameraIndex)
if not(videoCapture.isOpened()):
    raise Exception('could not connect to camera')


#prepare mask and define realworld map dimension
corner_mask_path = r"C:\Users\Carl\Desktop\Project\mask_corner.JPG"      #need picture
mask = cv2.imread(corner_mask_path)
dimension_paper = [118.9,84.1] #cm A0
dim = (int(dimension_paper[0]*2),int(dimension_paper[1]*2))
# Switching red and blue channels
mask[:, :, [0, 2]] = mask[:, :, [2, 0]]

##make map


#connect to the thymio,set the kinpapping distance to 10,the start position to (0,0) and the tolerance for equality to 1e-6
robotStatus=FSMHelper(th,10,np.array([0,0]),1e-6)

##generate pathplanning object from occupancy grid
#pathPlanner=pathPlaning(occupancyGrid.copy(),1,1)

In [ ]:
while(True):         #main execution loop
    ##read sensors 
    
    #----get robot position from camera
    
    #-----get odometrie data
    estimatedRobotPose,timeElapsed=robot_control.odometry(estimatedRobotPose,timeElapsed, robot_control.MAX_SPEED)
    
    #-----estimate current robot position
    
    
    
    #check if unexcpected obstacle is present
    robotStatus.obstacleDetected=not(all(sensorValues==0 for sensorValues in robotStatus.thymio["prox.horizontal"]))
    
    #make desicions and work with the collected data 
    stateToExecute=switch.get(currentState)
    futureState=stateToExecute(robotStatus)
    
    #doRobotControl here
    robotStatus.pathToFollow=path_following(p, robotStatus.pathToFollow, THREASHOLD = 0.5)
    
    #stopping robot if goal reached end programm
    if(robotStatus.doStop==True):
        robotStatus.thymio.set_var("motor.left.target", 0)
        robotStatus.thymio.set_var("motor.right.target", 0)
    if(robotStatus.goalReached==True):
        robotStatus.thymio.set_var("motor.left.target", 0)
        robotStatus.thymio.set_var("motor.right.target", 0)
        break
    currentState=futureState
    
cv2.destroyAllWindows() 
cv2.VideoCapture(cameraIndex).release()

# ........



# pathplanning

In [ ]:
##placeholder
pil_imgray = Image.open('Images/obstaclesTestMap.jpg').convert('LA')
img = np.array(list(pil_imgray.getdata(band=0)), float)
img.shape = (pil_imgray.size[1], pil_imgray.size[0])
img=img<200
occupancyGrid=img.astype(int)


##generate pathplanning object from occupancy grid
pathPlanner=pathPlaning(occupancyGrid.copy(),1,1)
goal=np.array([90,15])
start=np.array([10,75])
pathPlanner.setGoal(goal)
pathPlanner.setStart(start)
path=pathPlanner.getOptimizedPath()
plt.imshow(occupancyGrid)
plt.plot(path[0], path[1], marker="o", color = 'blue');
plt.show()